In [ ]:
import numpy as np
import pandas as pd
import re, os

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error

from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport

from ase.io import read
import copy
from tqdm import tqdm 


In [ ]:
#!unzip mof_cif_test

In [ ]:
Atom_list = ['H', 'B', 'C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'V', 'Cr', 'Ni', 'Cu',
       'Zn', 'Br', 'I', 'Ba']

def count_Atom(content,atom_list):
     dict = {}           
     for key in atom_list:
         new = []
         for line in content[22:]:
            
             if key in line:
                
                 a = str(line.split()[0])
                 new.append(a)

             if "loop_" in line:                
                 dict[str("cif_"+key)] = len(new)

                 break
     return dict

#Metal = {'Mg':47.2,'V':53.5,'Cr':32.9,'Co':37.2,'Ni':37.7,'Cu':25.1,'Zn':36.6,}

def read_cif_toAtom(cifDir,Atom_list):
    list_out = []
    index_out = []
    for cifname in tqdm(os.listdir(cifDir), desc = 'Filename'):
        #delete .csv from filename and append to list "index_out"
        index_out.append(cifname[:-4])
        content = open(os.path.join(cifDir, cifname), 'r').readlines()
        list_out.append(count_Atom(content,Atom_list))

    data_atom = pd.DataFrame.from_dict(list_out)
    data_atom.index = index_out
    return data_atom

In [ ]:
cifDir = ['mof_cif_test','mof_cif_train',]
print(cifDir)
data_atom = []
for index, cifdir in enumerate(cifDir):
    data_atom.append(read_cif_toAtom(cifdir,Atom_list))
    

['mof_cif_test', 'mof_cif_train']
Filename: 100%|██████████| 68613/68613 [01:29<00:00, 762.46it/s]


In [ ]:
train_data_full = pd.read_csv('tmlcc-2021/train.csv', index_col='MOFname')
test_data_full = pd.read_csv('tmlcc-2021/test.csv', index_col='MOFname')

In [ ]:
train_data_full.head()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
MOFname,,,,,,,,,,,,,
mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,pcu,22.864166,6.786041,105.284502
mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,etb,33.616780,7.147286,101.224774
mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,pcu,19.263726,6.347967,118.987011
mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,sra,25.701377,6.190085,187.626004
mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,pcu,30.001838,6.478063,79.210001


In [ ]:
test_data_edit = pd.concat([test_data_full,data_atom[0]],axis=1) 
train_data_edit = pd.concat([train_data_full,data_atom[1]],axis=1) 


In [ ]:
test_data_edit.reset_index().rename(columns={'index':'MOFname'})
train_data_edit.reset_index().rename(columns={'index':'MOFname'})

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,cif_S,cif_Cl,cif_V,cif_Cr,cif_Ni,cif_Cu,cif_Zn,cif_Br,cif_I,cif_Ba
0,mof_unit_1,1116.667429,875.240600,0.00,0.07899,0.0607,COOH-OEt,3,4,11,...,0,0,0,0,0,0,2,0,0,0
1,mof_unit_2,2769.503842,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,0,0,0,0,0,0,0,0,0,2
2,mof_unit_3,1089.818728,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,0,0,0,0,0,2,0,0,0,0
3,mof_unit_4,2205.198301,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,1,0,4,0,0,0,0,0,0,0
4,mof_unit_5,1137.800963,901.736120,0.00,0.07778,0.0591,NHMe-OH,2,1,22,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,mof_unit_68609,1188.302573,1001.700216,0.00,0.00000,0.0000,Pr-F,3,4,24,...,0,0,0,0,0,0,2,0,0,0
68609,mof_unit_68610,1506.660363,1493.296496,0.00,0.01108,0.0000,SO3H,10,42,46,...,1,0,0,0,0,0,0,0,0,2
68610,mof_unit_68611,2035.532738,1959.518320,0.00,0.00000,0.0000,OPr,4,14,22,...,0,0,0,6,0,0,0,0,0,0
68611,mof_unit_68612,3985.426053,3638.677280,0.00,0.00000,0.0000,OPr-Me,4,4,15,...,0,0,0,12,0,0,0,0,0,0


In [ ]:
test_data_edit.reset_index().rename(columns={'index':'MOFname'}).to_csv('tmlcc-feated/test.csv', index = False)
train_data_edit.reset_index().rename(columns={'index':'MOFname'}).to_csv('tmlcc-feated/train.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4dd58037-7658-4771-aa43-d87d2faa788a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>